In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import ToPILImage, Grayscale

In [ ]:
class CustomMNISTDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform = None, train=True):
        self.mnist_data = datasets.MNIST(root=root, train=train, transform=transform, download=True)
        self.train = train
        self.resize_transform = transforms.Resize((224, 224))

        if train:
            self.filtered_indices = [idx for idx, label in enumerate(self.mnist_data.targets) if label < 6]
        else:
            self.filtered_indices = [6 if label >= 6 else label for _, label in self.mnist_data]
        #self.transform = transform
        #self.to_pil = ToPILImage()
        #self.grayscale = Grayscale(num_output_channels=3)

    def __len__(self):
        return len(self.filtered_indices)

    def __getitem__(self, idx):
        if self.train:
            filtered_idx = self.filtered_indices[idx]
            image, label = self.mnist_data[filtered_idx]
        else:
            image, _ = self.mnist_data[idx]
            label = self.filtered_indices[idx]



        return image, label


In [ ]:
# Define the transformations
#transform = transforms.Compose([transforms.ToTensor()])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: torch.cat([x, x, x], 0)),
                #transforms.Normalize(mean, std),
                 ])
## train_test dataset
train_dataset = CustomMNISTDataset(root='path_to_mnist_dataset12', transform=transform, train=True)
test_dataset = CustomMNISTDataset(root='path_to_mnist_dataset12', transform=transform, train=False)


## Train_Testest Dataloader
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 240618282.88it/s]

Extracting path_to_mnist_dataset12/MNIST/raw/train-images-idx3-ubyte.gz to path_to_mnist_dataset12/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 92682244.70it/s]


Extracting path_to_mnist_dataset12/MNIST/raw/train-labels-idx1-ubyte.gz to path_to_mnist_dataset12/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 73635197.63it/s]

Extracting path_to_mnist_dataset12/MNIST/raw/t10k-images-idx3-ubyte.gz to path_to_mnist_dataset12/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 25919086.76it/s]


Extracting path_to_mnist_dataset12/MNIST/raw/t10k-labels-idx1-ubyte.gz to path_to_mnist_dataset12/MNIST/raw



In [ ]:
from torch.optim import optimizer
from torch.distributions import Beta
import tempfile
import random
import os
batch_size = 32

In [ ]:
class myModel(nn.Module):
  def __init__(self):
    super(myModel,self).__init__()
    self.feat_extractor = models.resnet18(weights = 'IMAGENET1K_V1')
    self.feat_extractor.fc = nn.Identity()
    self.classifier = nn.Linear(512,7)
  def forward(self,img,label):
    feat = self.feat_extractor(img)
    feat_new=torch.zeros_like(feat).to(device)
    for i in range(img.shape[0]):
        #for j in range(5):
        lambda_ = Beta(torch.FloatTensor([2]), torch.FloatTensor([2])).sample()
        lambda_ = lambda_.to(device)
        # batch_j =[]
        index1 = random.randint(0, img.shape[0] -1)
        index2 = random.randint(0, img.shape[0] - 1)
        # batch_j.append(img[index1])
        # batch_j.append(img[index2])
        mixup = lambda_ * feat[index1] + (1 - lambda_) * feat[index2]
        # print(mixup.shape)
        #x_midlayer = myModel().feat_extractor.fc(mixup)
        #x_new = myModel().classifier(x_midlayer)
        feat_new[i,:] = mixup
    ####
    feat_final = torch.cat((feat,feat_new),axis=0)
    prob = self.classifier(feat_final)
    return prob
#def generate_data_placeholder(model, batch_size, dataset):
    #mixed_outputs = []
    #for _ in range(batch_size):
        #lambda_ = Beta(torch.FloatTensor([2]), torch.FloatTensor([2])).sample()
       # #index1 = random.randint(0, len(dataset) - 1)
        #index2 = random.randint(0, len(dataset) - 1)
        #img1, _ = dataset[index1]
        #img2, _ = dataset[index2]
        #img1 = img1.unsqueeze(0)
        #img2 = img2.unsqueeze(0)
        #mixup = lambda_ * model.feat_extractor(img1) + (1 - lambda_) * model.feat_extractor(img2)
        #x_midlayer = model.feat_extractor.fc(mixup)
        #x_new = model.classifier(x_midlayer)
        #mixed_outputs.append(x_new)
   # mixed_outputs = torch.cat(mixed_outputs, dim=0)
   # return mixed_outputs

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = myModel().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
dataloaders = {
    'train': train_data_loader,
    'val': test_data_loader
}
dataset_sizes = {
    'train': len(train_data_loader),
    'val':len(test_data_loader)
}

batch_size = 32

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # print(newlabel.shape)
    # Create a temporary directory to save training checkpoints
    with tempfile.TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    newlabel = torch.ones(inputs.shape[0]) * 6
                    # print("NewLable",newlabel.shape)

                    newlabel = newlabel.to(device)
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        # print(labels.shape)
                        # print(newlabel.shape)
                        label_update = torch.cat((labels,newlabel),axis = 0).type(torch.LongTensor)
                        label_update=label_update.to(device)
                        # print(label_update.shape)
                        # print(inputs.shape)
                        outputs = model(inputs,labels)
                        # print("outputs",outputs.shape)

                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, label_update)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == label_update.data)
                #if phase == 'train':
                   # scheduler.step()

                epoch_loss = running_loss / (dataset_sizes[phase]*2*batch_size)
                epoch_acc = running_corrects.double() / (dataset_sizes[phase]*2*batch_size)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
trainresult = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.2283 Acc: 0.8323
val Loss: 0.2411 Acc: 0.7585

Epoch 1/9
----------
train Loss: 0.1683 Acc: 0.8787
val Loss: 0.2719 Acc: 0.7159

Epoch 2/9
----------
train Loss: 0.1590 Acc: 0.8852
val Loss: 0.2330 Acc: 0.7473

Epoch 3/9
----------
train Loss: 0.1530 Acc: 0.8878
val Loss: 0.2488 Acc: 0.7250

Epoch 4/9
----------
train Loss: 0.1473 Acc: 0.8927
val Loss: 0.2230 Acc: 0.7536

Epoch 5/9
----------
train Loss: 0.1470 Acc: 0.8929
val Loss: 0.2535 Acc: 0.7195

Epoch 6/9
----------
train Loss: 0.1434 Acc: 0.8954
val Loss: 0.2159 Acc: 0.7681

Epoch 7/9
----------
train Loss: 0.1422 Acc: 0.8952
val Loss: 0.2615 Acc: 0.7036

Epoch 8/9
----------
train Loss: 0.1397 Acc: 0.8980
val Loss: 0.2650 Acc: 0.7042

Epoch 9/9
----------
train Loss: 0.1406 Acc: 0.8962
val Loss: 0.2414 Acc: 0.7342

Training complete in 8m 20s
Best val Acc: 0.768071
